In [31]:
from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.static import players
from nba_api.stats.static import teams 
from nba_api.live.nba.endpoints import scoreboard
from multiprocessing import Pool
from concurrent.futures import ProcessPoolExecutor
from joblib import Parallel, delayed
import pandas as pd

In [1]:


all_team_ids = [teams.teams[i][0] for i in range(len(teams.teams))]
all_team_names = [teams.teams[i][5] for i in range(len(teams.teams))]
team_id_dict = {i:j for i,j in zip(all_team_ids, all_team_names)}

In [16]:
team_id_dict

{1610612737: 'Atlanta Hawks',
 1610612738: 'Boston Celtics',
 1610612739: 'Cleveland Cavaliers',
 1610612740: 'New Orleans Pelicans',
 1610612741: 'Chicago Bulls',
 1610612742: 'Dallas Mavericks',
 1610612743: 'Denver Nuggets',
 1610612744: 'Golden State Warriors',
 1610612745: 'Houston Rockets',
 1610612746: 'Los Angeles Clippers',
 1610612747: 'Los Angeles Lakers',
 1610612748: 'Miami Heat',
 1610612749: 'Milwaukee Bucks',
 1610612750: 'Minnesota Timberwolves',
 1610612751: 'Brooklyn Nets',
 1610612752: 'New York Knicks',
 1610612753: 'Orlando Magic',
 1610612754: 'Indiana Pacers',
 1610612755: 'Philadelphia 76ers',
 1610612756: 'Phoenix Suns',
 1610612757: 'Portland Trail Blazers',
 1610612758: 'Sacramento Kings',
 1610612759: 'San Antonio Spurs',
 1610612760: 'Oklahoma City Thunder',
 1610612761: 'Toronto Raptors',
 1610612762: 'Utah Jazz',
 1610612763: 'Memphis Grizzlies',
 1610612764: 'Washington Wizards',
 1610612765: 'Detroit Pistons',
 1610612766: 'Charlotte Hornets'}

In [29]:
from nba_api.stats.endpoints import leaguegamefinder


gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=1610612739)
games = gamefinder.get_data_frames()[0]

#  make an array of dataframes for all games
dict_all_games = {team:leaguegamefinder.LeagueGameFinder(team_id_nullable=id).get_data_frames()[0] for team, id in zip(all_team_names,all_team_ids)}

In [32]:
all_games_df = pd.concat(list(dict_all_games.values()), axis=0)

In [28]:
dict_all_games[0]['Atlanta Hawks'].columns

Index(['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'PTS', 'FGM', 'FGA', 'FG_PCT',
       'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB',
       'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS'],
      dtype='object')

In [39]:
all_games_df['GAME_DATE'].astype(str)

0       2024-04-17
1       2024-04-14
2       2024-04-12
3       2024-04-10
4       2024-04-09
           ...    
2967    1988-11-12
2968    1988-11-11
2969    1988-11-08
2970    1988-11-05
2971    1988-11-04
Name: GAME_DATE, Length: 105744, dtype: object

In [42]:
# find games by year
def find_games_by_year(year):
    games_year = all_games_df[all_games_df['GAME_DATE'].str.contains(str(year))]
    return games_year
    

find_games_by_year(2014)

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
924,22014,1610612737,ATL,Atlanta Hawks,0021400466,2014-12-30,ATL vs. CLE,W,239,109,...,0.731,7.0,37.0,44.0,23,13.0,5,15,19,8.0
925,22014,1610612737,ATL,Atlanta Hawks,0021400449,2014-12-27,ATL @ MIL,W,238,90,...,0.789,17.0,34.0,51.0,25,9.0,4,11,17,5.0
926,22014,1610612737,ATL,Atlanta Hawks,0021400434,2014-12-26,ATL vs. MIL,L,240,77,...,0.786,8.0,30.0,38.0,20,8.0,4,22,19,-30.0
927,22014,1610612737,ATL,Atlanta Hawks,0021400420,2014-12-23,ATL vs. LAC,W,240,107,...,0.771,9.0,34.0,43.0,28,6.0,3,14,21,3.0
928,22014,1610612737,ATL,Atlanta Hawks,0021400413,2014-12-22,ATL @ DAL,W,240,105,...,0.667,6.0,37.0,43.0,26,6.0,6,17,21,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
956,22013,1610612766,CHA,Charlotte Bobcats,0021300535,2014-01-10,CHA @ MIN,L,243,92,...,0.714,4.0,29.0,33.0,23,10.0,7,10,17,-27.0
957,22013,1610612766,CHA,Charlotte Bobcats,0021300508,2014-01-07,CHA vs. WAS,L,240,83,...,0.625,3.0,35.0,38.0,19,3.0,1,13,13,-14.0
958,22013,1610612766,CHA,Charlotte Bobcats,0021300497,2014-01-04,CHA @ SAC,W,240,113,...,0.609,6.0,32.0,38.0,22,5.0,4,11,19,10.0
959,22013,1610612766,CHA,Charlotte Bobcats,0021300480,2014-01-02,CHA @ POR,L,240,104,...,0.806,6.0,24.0,30.0,15,10.0,2,9,10,-30.0


In [46]:
def find_games_by_team(team_str):
    games_team = all_games_df[all_games_df['TEAM_NAME'].str.contains(team_str)]
    return games_team

find_games_by_team('Hawks')

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,52023,1610612737,ATL,Atlanta Hawks,0052300111,2024-04-17,ATL @ CHI,L,241,116,...,0.821,8.0,26.0,34.0,30,4.0,2,9,16,-15.0
1,22023,1610612737,ATL,Atlanta Hawks,0022301188,2024-04-14,ATL @ IND,L,241,115,...,0.926,9.0,23.0,32.0,25,6.0,5,15,12,-42.0
2,22023,1610612737,ATL,Atlanta Hawks,0022301178,2024-04-12,ATL @ MIN,L,240,106,...,0.680,9.0,31.0,40.0,23,4.0,1,14,25,-3.0
3,22023,1610612737,ATL,Atlanta Hawks,0022301159,2024-04-10,ATL vs. CHA,L,240,114,...,0.625,7.0,31.0,38.0,35,7.0,2,16,20,-1.0
4,22023,1610612737,ATL,Atlanta Hawks,0022301147,2024-04-09,ATL vs. MIA,L,292,111,...,0.786,17.0,42.0,59.0,28,13.0,2,15,23,-6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3691,21983,1610612737,ATL,Atlanta Hawks,0028300058,1983-11-06,ATL @ MIL,L,240,84,...,0.667,11.0,26.0,37.0,18,4.0,5,14,24,NaN
3692,21983,1610612737,ATL,Atlanta Hawks,0028300041,1983-11-04,ATL vs. CHI,W,240,103,...,0.692,19.0,27.0,46.0,31,14.0,13,18,27,NaN
3693,21983,1610612737,ATL,Atlanta Hawks,0028300027,1983-11-01,ATL vs. WAS,W,240,95,...,0.633,12.0,29.0,41.0,20,7.0,10,16,34,NaN
3694,21983,1610612737,ATL,Atlanta Hawks,0028300014,1983-10-29,ATL vs. DET,W,240,117,...,0.633,27.0,21.0,48.0,28,14.0,7,23,35,NaN


In [66]:
def find_games_by_matchup(team_1, team_2):
    team_1 = team_1.lower()
    team_2 = team_2.lower()
    games_matchup = all_games_df[(all_games_df['MATCHUP'].str.contains(team_1.upper())) & (all_games_df['MATCHUP'].str.contains(team_2.upper()))]

    return games_matchup

find_games_by_matchup('cle', 'LAL')

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
13,22023,1610612739,CLE,Cleveland Cavaliers,0022301127,2024-04-06,CLE @ LAL,L,243,97,...,0.667,11.0,29.0,40.0,24,11.0,3,18,11,-19.0
75,22023,1610612739,CLE,Cleveland Cavaliers,0022300242,2023-11-25,CLE vs. LAL,L,241,115,...,0.870,7.0,30.0,37.0,30,13.0,4,14,16,-6.0
163,22022,1610612739,CLE,Cleveland Cavaliers,0022200360,2022-12-06,CLE vs. LAL,W,241,116,...,0.765,9.0,35.0,44.0,25,11.0,3,7,20,14.0
179,22022,1610612739,CLE,Cleveland Cavaliers,0022200140,2022-11-06,CLE @ LAL,W,240,114,...,0.889,9.0,39.0,48.0,19,7.0,6,12,20,14.0
209,22021,1610612739,CLE,Cleveland Cavaliers,0022101072,2022-03-21,CLE vs. LAL,L,240,120,...,0.842,8.0,30.0,38.0,29,3.0,2,12,19,-11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3751,21985,1610612747,LAL,Los Angeles Lakers,0028500061,1985-11-05,LAL vs. CLE,L,240,111,...,0.750,16.0,27.0,43.0,26,3.0,5,15,25,NaN
3794,21984,1610612747,LAL,Los Angeles Lakers,0028400731,1985-03-09,LAL vs. CLE,W,240,133,...,0.667,14.0,40.0,54.0,46,8.0,9,22,31,NaN
3837,21984,1610612747,LAL,Los Angeles Lakers,0028400215,1984-12-04,LAL @ CLE,W,240,116,...,0.842,17.0,32.0,49.0,25,7.0,10,19,18,NaN
3929,21983,1610612747,LAL,Los Angeles Lakers,0028300364,1984-01-04,LAL @ CLE,W,240,106,...,0.750,14.0,28.0,42.0,26,8.0,4,19,25,NaN


In [11]:
def find_games

{'gameDate': '2024-05-09',
 'leagueId': '00',
 'leagueName': 'National Basketball Association',
 'games': [{'gameId': '0042300202',
   'gameCode': '20240509/CLEBOS',
   'gameStatus': 2,
   'gameStatusText': 'Q2 3:05',
   'period': 2,
   'gameClock': 'PT03M05.00S',
   'gameTimeUTC': '2024-05-09T23:00:00Z',
   'gameEt': '2024-05-09T19:00:00-04:00',
   'regulationPeriods': 4,
   'ifNecessary': False,
   'seriesGameNumber': 'Game 2',
   'gameLabel': None,
   'gameSubLabel': None,
   'seriesText': 'BOS leads 1-0',
   'seriesConference': 'East',
   'poRoundDesc': 'Conf. Semifinals',
   'gameSubtype': '',
   'homeTeam': {'teamId': 1610612738,
    'teamName': 'Celtics',
    'teamCity': 'Boston',
    'teamTricode': 'BOS',
    'wins': 1,
    'losses': 0,
    'score': 51,
    'seed': 1,
    'inBonus': '0',
    'timeoutsRemaining': 6,
    'periods': [{'period': 1, 'periodType': 'REGULAR', 'score': 24},
     {'period': 2, 'periodType': 'REGULAR', 'score': 27},
     {'period': 3, 'periodType': 'REGU